In [1]:
# Importo las librerias
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
from sklearn import linear_model
import re

## Desafío DOS.
##### Introducción.
En este desafío desarrollaremos y evaluaremos el rendimiento de varios modelos para analizar su potencia y su poder de generalización,sobre datos referentes al mercado inmobiliario proporcionados por Properatti.

##### Modificación de datos y preprocesamiento previo.
Para continuar con el desafio número dos y desarrollar un modelo que nos permita inferir el precio por metro cuadrado, decidimos crear nuevas features con el objetivo de aumentar la cantidad de variables explicativas y lograr un buen ajuste para el modelo. Para ello utilizamos la técnica Regex para extraer información de la columna "description" y dummificar ciertas variables como las amenities.

In [2]:
#Importamos el dataset final del desafío uno para mergearlo con el dataset original de properatti y rescatar la columna "description"
df_original= pd.read_csv("./Data/properatti.csv")
df_final_desafio_1= pd.read_csv("./Data/Data_final_desafio_1.csv")
display(df_final_desafio_1.head(3),
       df_original.head(3))

,Unnamed: 0,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,Provincia,Barrio,property_title,propiedad_PH,propiedad_apartment,propiedad_house,propiedad_store,precio_M2_total,precio_M2_covered
0,0,62000.0,55.0,40.0,Capital Federal,Mataderos,CASA,1,0,0,0,1127.27,1550.000000
1,2,72000.0,55.0,55.0,Capital Federal,Mataderos,NaN,0,1,0,0,1309.09,1309.090909
2,4,64000.0,35.0,35.0,Buenos Aires Costa Atlántica,Mar del Plata,NaN,0,1,0,0,1828.57,1828.571429


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [3]:
#Eliminamos columnas del df original de properatti para mergear con el df final del desafio 1 y realizar el regex.
all_cols=df_original.columns.values.tolist()

In [4]:
cols_deseadas= ['Unnamed: 0','description',"property_type","rooms"]
cols_no_deseadas= [x for x in all_cols if x not in cols_deseadas]
df_original.drop(cols_no_deseadas, axis=1, inplace=True)

In [5]:
df_original.columns

Index(['Unnamed: 0', 'property_type', 'rooms', 'description'], dtype='object')

In [6]:
#Mergeamos para agregar la columna description.
df_desafio_1_y_df_original= df_final_desafio_1.merge(df_original, on="Unnamed: 0", how="inner")

In [7]:
#Chequeamos que este todo bien.
print((df_desafio_1_y_df_original.shape))
print((df_desafio_1_y_df_original.columns))

(71827, 16)
Index(['Unnamed: 0', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'Provincia', 'Barrio', 'property_title',
       'propiedad_PH', 'propiedad_apartment', 'propiedad_house',
       'propiedad_store', 'precio_M2_total', 'precio_M2_covered',
       'property_type', 'rooms', 'description'],
      dtype='object')


## Feature Engineering
#### Utilizamos Regex para extraer información de la columna "description".

##### Utilizando dicha técnica vamos a dummificar ciertas amenities. "Pileta, Gimnasio, Vista, Patio y Garaje"

##### Pileta

In [8]:
#Estandarizamos la serie.
serie_description= df_desafio_1_y_df_original.description.apply(lambda x: str(x).upper())
serie_description

0        2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...
1        2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...
2        DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...
3        MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...
4        EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...
                               ...                        
71822    EXCELENTE UBICACIÓN - AVENIDA PUEYRREDON Y LAS...
71823    EDIFICIO EN ESQUINA DE MUY BUENA CATEGORÍA DE ...
71824    EXCELENTE E IMPECABLE CASA EN VENTA EN LAS LOM...
71825    VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...
71826    2 AMB AL CONTRAFRENTE,  LUMINOSO. EL DEPARTAME...
Name: description, Length: 71827, dtype: object

In [9]:
#Definimos el patrón y compilamos.
pattern = "(?P<pileta_ref> PILETA|PISCINA|JACUZZI|YACUSI +$)"
pattern_compilado= re.compile(pattern)
buscar_match= serie_description.apply(lambda x: pattern_compilado.search(x))
matcheados =buscar_match.notnull()

In [10]:
#Tienen pileta.
matcheados.sum()

20037

In [11]:
#Creamos una columna con los datos relevados.
df_desafio_1_y_df_original['PILETA'] = buscar_match[matcheados].apply(lambda x: x if x is np.NaN else x.group("pileta_ref"))

##### Gimnasio

In [12]:
#Compilamos el patron.
pattern_gym= "(?P<gym_ref> GIMNASIO|GYM|GIMNASIA|GIMNACIO|ENTRENAR|ENTRENAMIENTO|FITNESS +$)"
compile_pattern_gym= re.compile(pattern_gym)
matchear= serie_description.apply(lambda x: compile_pattern_gym.search(x))
mask_no_nulos_gym= matchear.notnull()
mask_no_nulos_gym.sum()
#Encontramos 6260 entradas que indican que poseen gimnasio.

6260

In [13]:
#Extraemos esos valores y los volcamos en una columna.
extraer_match= matchear[mask_no_nulos_gym].apply(lambda x: x if x is np.NaN else x.group("gym_ref"))
df_desafio_1_y_df_original["GYM"]= extraer_match

##### Patio

In [14]:
#Compilamos el patron.
pattern_patio= "(?P<patio_ref> TERRAZA|BALCON+$)"
patron_compilado_patio= re.compile(pattern_patio)
matchear_patio= serie_description.apply(lambda x: patron_compilado_patio.search(x))
encontrados_patio=matchear_patio.notnull()

In [15]:
#Cantidad de entradas con terraza y balcon encontrados.
encontrados_patio.sum()

15316

In [16]:
extraer_match_patio= matchear_patio[encontrados_patio].apply(lambda x: x if x is np.NaN else x.group("patio_ref"))
df_desafio_1_y_df_original["PATIO"]= extraer_match_patio

##### Luminosidad

In [17]:
#Compilamos el patron.
pattern_luz= "(?P<luz_ref> LUZ|LUMINOSO|SOL|ILUMINADO|LUMINOCIDAD|SOLAR|VISTA +$)"
patron_compilado_luz= re.compile(pattern_luz)
matchear_luz= serie_description.apply(lambda x: patron_compilado_luz.search(x))
encontrados_luz=matchear_luz.notnull()

In [18]:
#Cantidad de entradas encontradas con dichas características.
encontrados_luz.sum()

29655

In [19]:
extraer_match_luz= matchear_luz[encontrados_luz].apply(lambda x: x if x is np.NaN else x.group("luz_ref"))
df_desafio_1_y_df_original["LUZ"]= extraer_match_luz

##### Garaje

In [20]:
# Creamos el patron para Garaje.
garaje_pattern = "(?P<garage_ref> GARAGE|COCHERA|PARKING|GARAJE|ESTACIONAMIENTO +$)"
garaje_pattern_regex = re.compile(garaje_pattern)
matchear_garaje= serie_description.apply(lambda x: garaje_pattern_regex.search(x))
encontrados_garaje=matchear_garaje.notnull()

In [21]:
#Cantidad de entradas encontradas con garaje.
encontrados_garaje.sum()

31631

In [22]:
extraer_match_garaje= matchear_garaje[encontrados_garaje].apply(lambda x: x if x is np.NaN else x.group("garage_ref"))
df_desafio_1_y_df_original["GARAJE"]= extraer_match_garaje

In [23]:
#Chequeamos que esten creadas las columnas.
df_desafio_1_y_df_original.head(3)

,Unnamed: 0,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,Provincia,Barrio,property_title,propiedad_PH,propiedad_apartment,propiedad_house,...,precio_M2_total,precio_M2_covered,property_type,rooms,description,PILETA,GYM,PATIO,LUZ,GARAJE
0,0,62000.0,55.0,40.0,Capital Federal,Mataderos,CASA,1,0,0,...,1127.27,1550.000000,PH,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",NaN,NaN,NaN,NaN,NaN
1,2,72000.0,55.0,55.0,Capital Federal,Mataderos,NaN,0,1,0,...,1309.09,1309.090909,apartment,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,NaN,NaN,NaN,NaN,NaN
2,4,64000.0,35.0,35.0,Buenos Aires Costa Atlántica,Mar del Plata,NaN,0,1,0,...,1828.57,1828.571429,apartment,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,NaN,NaN,NaN,NaN,NaN


## Dummificamos las variables encontradas.

In [24]:
df_desafio_1_y_df_original["Pileta_dummie"]=df_desafio_1_y_df_original.PILETA.apply(lambda x : 0 if x is np.NaN else 1)
df_desafio_1_y_df_original["Gym_dummie"]= df_desafio_1_y_df_original.GYM.apply(lambda x: 0 if x is np.NaN else 1)
df_desafio_1_y_df_original["Luz_dummie"]= df_desafio_1_y_df_original.LUZ.apply(lambda x: 0 if x is np.NaN else 1)
df_desafio_1_y_df_original["Patio_dummie"]= df_desafio_1_y_df_original.PATIO.apply(lambda x: 0 if x is np.NaN else 1)
df_desafio_1_y_df_original["Garage_dummie"]= df_desafio_1_y_df_original.GARAJE.apply(lambda x: 0 if x is np.NaN else 1)
df_desafio_1_y_df_original.drop(["PILETA","GYM","PATIO","LUZ","GARAJE","property_title"], axis=1, inplace=True)

In [25]:
df_desafio_1_y_df_original.head(3)

,Unnamed: 0,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,Provincia,Barrio,propiedad_PH,propiedad_apartment,propiedad_house,propiedad_store,precio_M2_total,precio_M2_covered,property_type,rooms,description,Pileta_dummie,Gym_dummie,Luz_dummie,Patio_dummie,Garage_dummie
0,0,62000.0,55.0,40.0,Capital Federal,Mataderos,1,0,0,0,1127.27,1550.000000,PH,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",0,0,0,0,0
1,2,72000.0,55.0,55.0,Capital Federal,Mataderos,0,1,0,0,1309.09,1309.090909,apartment,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,0,0,0,0,0
2,4,64000.0,35.0,35.0,Buenos Aires Costa Atlántica,Mar del Plata,0,1,0,0,1828.57,1828.571429,apartment,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,0,0,0,0,0


In [26]:
#Exportamos el df como CSV para trabajarlo aparte con los modelos.
ubicación= r"C:\Users\Carlos\Desktop\Desafio 2\Data\DATA_VA_1.csv"
df_desafio_1_y_df_original.to_csv(ubicación)